## Define Azure AI Agent for Chainlit

## Step 1: Define Tools Functions

In [47]:
import json
from typing import Any, Callable, Set, Dict, List, Optional

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {
        "New York": "Sunny, 25°C", 
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
    }
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

def fetch_restaurant(location: str) -> str:
    """
    Fetches the restaurant information for the specified location.

    :param location (str): The location to fetch the restaurant for.
    :return: Restaurant information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a restaurant API.
    # Here, we'll mock the response.
    mock_restaurant_data = {
        "New York": "Tatiana by Kwame Onwuachi, Katz’s Delicatessen, Peter Luger Steakhouse, Sylvia's, Nathan's Famous", 
        "London": "St. JOHN, Señor Ceviche, Gloria and Circolo Popolare, Normah's, Bouchon Racine", 
        "Tokyo": "Chanko & Wanko Restaurant Asakusa Sumo Club, Sky Restaurant 634 Musashi, Ichiran, Shibuya, Rokkasen Otakibashiidori, Hakushu - Kobe Teppanyaki"
    }
    restaurant = mock_restaurant_data.get(location, "Restaurant data not available for this location.")
    restaurant_json = json.dumps({"restaurant": restaurant})
    return restaurant_json

def fetch_budget() -> str:
    """
    Fetches the budget information for the specified location.
    :return: budget information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a another API.
    # Here, we'll mock the response.
    mock_budget_data = {
        "New York": """
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "London": """
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "Tokyo": """
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
    }
    budget_json = json.dumps({"budget": mock_budget_data})
    return budget_json


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather, fetch_restaurant, fetch_budget
}

## Step 2: Create an Azure AI Client

In [31]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import AzureAISearchTool, AzureAISearchQueryType
from azure.ai.projects.models import FunctionTool, ToolSet, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# HostName can be found by navigating to your discovery_url and removing the leading "https://" and trailing "/discovery" 
# To find your discovery_url, run the CLI command: az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url
# Project Connection example: eastus.api.azureml.ms;my-subscription-id;my-resource-group;my-hub-name

connection_string = os.environ["PROJECT_CONNECTION_STRING"] 

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string,
)

## Step 3: Setup the AzureAISearchTool

query type values can be found here: https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.models.azureaisearchquerytype

In [33]:
# Initialize agent AI search tool and add the search index connection ID and index name
connection_id = os.getenv("PROJECT_CONNECTION_ID_AZURE_AI_SEARCH")
index_name = "travel-product-index"
ai_search = AzureAISearchTool(
    index_connection_id=connection_id, 
    index_name=index_name,
    query_type=AzureAISearchQueryType.VECTOR_SEMANTIC_HYBRID,
    top_k=5,
)

## Step 4: Create the Agent

In [34]:
# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
toolset.add(ai_search)
    
# Create a new agent with the toolset
agent = project_client.agents.create_agent(
    model="gpt-4o", 
    name="my-chainlit-agent", 
    instructions="""
        You are an AI Travel Agent. 
        You will answer questions about travel based on the tools provided.
        When asked questions about products, you will use the Azure AI Search tool to find relevant products.
    """, 
    toolset=toolset
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_X6RXsMsspn6QVdYDeZVT9OkO


## Helper Function

In [35]:
def run_agent(user_input):  
    # Step 3: Add a message to the thread  
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )
    print(f"Created message, ID: {message.id}")

    # Step 4 & 5: Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Step 6: Display the Agent's Response
    elif run.status == 'completed':
            # Fetch all messages in the thread
            messages = project_client.agents.list_messages(thread_id=thread.id)
            if messages.data:
                agent_message = messages.data[0]  # Get the last assistant message
                print(f"Agent Response: {agent_message.content[0].text.value}") 
            else:
                print("No messages found.")

## Test Agent

In [36]:
# Create thread
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_HEw1dMkLvrZuaKNxiEVqAEmY


In [37]:
user_input = "Hello, what is the weather in Tokyo?"
run_agent(user_input)

Created message, ID: msg_xpiTJNMDIqYoF0vFfvSu4ByK
Agent Response: The current weather in Tokyo is rainy, with a temperature of 22°C.


In [38]:
user_input = "What is the budget there?"
run_agent(user_input)

Created message, ID: msg_VOMmD9Ux2ZihbYwF0UaflkGn
Agent Response: In Tokyo, the budget varies depending on your travel style:

- **Budget Travelers:** Around $100 per day, including staying in hostels, eating at budget restaurants, and using public transportation.
- **Mid-Range Travelers:** Approximately $286 per day, which covers mid-range hotels, dining at average restaurants, and visiting some attractions.
- **Luxury Travelers:** About $908 per day, including luxury hotels, fine dining, and private transportation.


In [39]:
print(project_client)

In [40]:
user_input = "Where can I eat in London?"
run_agent(user_input)

Created message, ID: msg_AXKYxjEPbZV8FjQ3C2nnrwxj
Agent Response: In London, you can dine at some wonderful places, including:

- **St. JOHN:** Known for traditional British cuisine.
- **Señor Ceviche:** Offering delicious Peruvian food.
- **Gloria and Circolo Popolare:** Famous for their Italian dishes.
- **Normah's:** Serving Malaysian cuisine.
- **Bouchon Racine:** Perfect spot for French classic meals.


In [12]:
user_input = "I need a travel lock and luggage. What can you recommend? Also how much will it cost?"
run_agent(user_input)

Created message, ID: msg_fJ5vrcPCzfqs1MCahj0AzYdV
Agent Response: Here are some recommendations for travel locks and luggage along with their prices:

1. **Travel Lock**:
   - **WanderSafe Travel Lock**: Priced at $25, it’s TSA-approved and made from durable zinc alloy for security. It features a resettable 3-digit combination lock and a flexible steel cable for easy attachment to luggage【17:0†source】.

2. **Luggage Options**:
   - **Nomad Traveler Suitcase**: Costing $275, this luggage is made from high-impact ABS plastic, has expandable capacity, spinner wheels, and a built-in TSA-Approved lock【17:1†source】.
   - **Voyager Pro Luggage**: Priced at $250, made from durable polycarbonate, and includes expandable packing space along with TSA-approved locks【17:2†source】.
   - **Globetrotter Elite Suitcase**: Costing $299, it’s constructed with premium material, expandable capacity, spinner wheels, and a lightweight design with TSA-approved locks【17:3†source】.

These options suit various n

In [13]:
user_input = "I only have 300 USD. What country can I visit for 4 days?"
run_agent(user_input)

Created message, ID: msg_rw27SvF7C9FmcFEtHWlazf5G
Agent Response: Here are some countries you could visit for 4 days within a $300 budget:

1. **Tokyo, Japan**:
   - Budget travelers can manage with $100/day, totaling $400 for 4 days. 
   - If you adjust your accommodation (e.g., hostels), it could be achievable slightly over $300.

2. **London, United Kingdom**:
   - Budget travelers can manage with $75/day, totaling $300 for 4 days.
   - Stick to budget accommodation, cook meals yourself, and use public transport.

3. **Closer Options for US Travelers**:
   - Mexico or Canada often offer budget options for travel, transportation, and food.

Let me know if you'd like detailed budget planning!


In [46]:
connection_string = os.environ["PROJECT_CONNECTION_STRING"] 
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string,
)
AGENT_ID = "asst_X6RXsMsspn6QVdYDeZVT9OkO"

# Create thread
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

user_input = "Hello, what is the weather in Tokyo?"
# Add a message to the thread  
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_input,
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=AGENT_ID)

Created thread, ID: thread_Yy2DHFqtYunrfScZeSaSYv9x
Created message, ID: msg_Hiiaynj954YZ0iW5d4vWF4Z5


ValueError: Toolset is not available in the client.

In [52]:
# Define the function to run the agent
def run_agent(user_input, project_client, thread_id):  

    print(agent.id)
    # Add a message to the thread  
    message = project_client.agents.create_message(
        thread_id=thread_id,
        role="user",
        content=user_input,
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread_id, agent_id=AGENT_ID)
    
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display the Agent's Response
    elif run.status == 'completed':
            # Fetch all messages in the thread
            messages = project_client.agents.list_messages(thread_id=thread_id)
            if messages.data:
                agent_response = messages.data[0]  # Get the last assistant message
                print(f"Agent Response: {agent_response.content[0].text.value}") 
            else:
                print("No messages found.")
    
    return agent_response

In [43]:
user_input = "What restaurants can you recommend in tokyo"
agent_response = run_agent(user_input, project_client, thread.id)
print(agent_response)

asst_X6RXsMsspn6QVdYDeZVT9OkO
Created message, ID: msg_svfx380g6zlBwEg7W0xtcgu6


ValueError: Toolset is not available in the client.

In [50]:
connection_string = os.environ["PROJECT_CONNECTION_STRING"] 

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string,
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
    
# Create a new agent with the toolset
agent = project_client.agents.create_agent(
    model="gpt-4o", 
    name="my-chainlit-agent", 
    instructions="""
        You are an AI Travel Agent. 
        You will answer questions about travel based on the tools provided.
        When asked questions about products, you will use the Azure AI Search tool to find relevant products.
    """
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_RidmabHRlMdtpVlPj7ch2LKM


In [55]:
connection_string = os.environ["PROJECT_CONNECTION_STRING"] 
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string,
)
AGENT_ID = "asst_RidmabHRlMdtpVlPj7ch2LKM"

# Create thread
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

user_input = "What is the capital of Japan?"
run_agent(user_input, project_client, thread.id)

Created thread, ID: thread_Ux7VIkwYjiWbU2j6hHodopJ9
asst_RidmabHRlMdtpVlPj7ch2LKM
Created message, ID: msg_npNarrZgER3ywQbncAgIM2LZ
Agent Response: The capital of Japan is **Tokyo**.


{'id': 'msg_40EkOCSA5IBkpDcbzBaTnGEs', 'object': 'thread.message', 'created_at': 1744994292, 'assistant_id': 'asst_RidmabHRlMdtpVlPj7ch2LKM', 'thread_id': 'thread_Ux7VIkwYjiWbU2j6hHodopJ9', 'run_id': 'run_bsDSGWt7rYt7Crk9kQLbsD63', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The capital of Japan is **Tokyo**.', 'annotations': []}}], 'attachments': [], 'metadata': {}}